In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
from src.data.sets import load_sets

X_train, y_train, X_val, y_val, X_test, y_test = load_sets()

print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)
print(X_test.shape)
print(y_test.shape)

(432274, 6)
(432274,)
(144092, 6)
(144092,)
(144092, 6)
(144092,)


### Define Architecture

Instantiate `PytorchMultiClass` with the correct number of input feature and save it into a variable called `model`
<p/>May need to adjust to take more inputs with beer api

In [3]:
from src.models.pytorch import PytorchMultiClass

# May need to adjust to take more inputs with beer api
# shape[1] refers to number of columns / features
model = PytorchMultiClass(X_train.shape[1], 104)

In [4]:
from src.models.pytorch import get_device

device = get_device()
model.to(device)

PytorchMultiClass(
  (layer_1): Linear(in_features=6, out_features=512, bias=True)
  (layer_2): Linear(in_features=512, out_features=128, bias=True)
  (layer_out): Linear(in_features=128, out_features=104, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.2, inplace=False)
  (batchnorm1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

### Train Model

In [10]:
def get_class_distribution(obj):
    items = X_train['beer_style']
    count = {}
    for item in items:
        count[item] = count.get(item, 0) + 1
    return count

In [7]:
#convert sets to PytorchDataset
from src.models.pytorch import PytorchDataset

train_dataset = PytorchDataset(torch.from_numpy(X_train).float(), torch.from_numpy(y_train).long())
val_dataset = PytorchDataset(torch.from_numpy(X_val).float(), torch.from_numpy(y_val).long())
test_dataset = PytorchDataset(torch.from_numpy(X_test).float(), torch.from_numpy(y_test).long())

#train_dataset = PytorchDataset(X=X_train, y=y_train)
#val_dataset = PytorchDataset(X=X_val, y=y_val)
#test_dataset = PytorchDataset(X=X_test, y=y_test)

In [11]:
target_list = []
for _, t in train_dataset:
    target_list.append(t)
    
target_list = torch.tensor(target_list)

In [12]:
class_count = [i for i in get_class_distribution(y_train).values()]
class_weights = 1./torch.tensor(class_count, dtype=torch.float) 
print(class_weights)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
class_weights_all = class_weights[target_list]

In [ ]:
criterion = nn.CrossEntropyLoss()

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

In [ ]:
N_EPOCHS = 5
BATCH_SIZE = 32

In [ ]:
from src.models.pytorch import train_classification, test_classification

for epoch in range(N_EPOCHS):
    train_loss, train_acc = train_classification(train_dataset, model=model, criterion=criterion, optimizer=optimizer, batch_size=BATCH_SIZE, device=device)
    valid_loss, valid_acc = test_classification(val_dataset, model=model, criterion=criterion, batch_size=BATCH_SIZE, device=device)

    print(f'Epoch: {epoch}')
    print(f'\t(train)\t|\tLoss: {train_loss:.4f}\t|\tAcc: {train_acc * 100:.1f}%')
    print(f'\t(valid)\t|\tLoss: {valid_loss:.4f}\t|\tAcc: {valid_acc * 100:.1f}%')

In [ ]:
torch.save(model, "../models/pytorch_multi_beer_style.pt")

In [ ]:
test_loss, test_acc = test_classification(test_dataset, model=model, criterion=criterion, batch_size=BATCH_SIZE, device=device)
print(f'\tLoss: {test_loss:.4f}\t|\tAccuracy: {test_acc:.1f}')